# Semantic Search on PDF Documents with qHNSW Index

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents.

<div class="alert alert-block alert-info">
<b>Tip:</b> This sample uses ‘qHNSW’ , a new vector index choice in KDB.AI. It will support the same API options as the existing ‘HNSW’ index but with the significant difference that the index is stored on-disk and memory-mapped as required. This means data inserts will have negligible memory and cpu footprints. The vector index can grow and be searched as long as there is disk space available. Among other cases, this stands out as a great index for memory contrained situations such as edge devices.
</div>

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Setup
1. Load PDF Data
2. KDB.AI Table Creation (qHNSW index)
3. LlamaIndex index & query_engine setup
4. Retrieve Similar Sentences & RAG
5. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
!pip install llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-readers-file llama-index-vector-stores-kdbai
!pip install pandas
!pip install kdbai_client


### Import Packages

In [52]:

import os
from getpass import getpass
import re
import os
import shutil
import time
import urllib

import pandas as pd

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.kdbai import KDBAIVectorStore

import kdbai_client as kdbai

OUTDIR = "pdf"
RESET = True

In [53]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads research paper pdf into your environment
if os.path.exists("./data/research_paper.pdf") == False:
  !mkdir ./data
  !wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

In [54]:
# OpenAI API Key: https://platform.openai.com/api
os.environ["OPENAI_API_KEY"] = (
    os.environ["OPENAI_API_KEY"]
    if "OPENAI_API_KEY" in os.environ
    else getpass("OpenAI API Key: ")
)

In [55]:
# Set up LlamaIndex Parameters

EMBEDDING_MODEL  = "text-embedding-3-small"
GENERATION_MODEL = 'gpt-4o-mini'

llm = OpenAI(model=GENERATION_MODEL)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)

Settings.llm = llm
Settings.embed_model = embed_model

### Configure Console

In [56]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [57]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We LlamaIndex SimpleDirectorReader to read in our PDF file.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [58]:
reader = SimpleDirectoryReader(
    input_dir="data",
)
documents = reader.load_data()

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [59]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [60]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

Compatibility with the KDB.AI server is not guaranteed.


##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [61]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns.

At this point you will select the index and metric you want to use for searching.

In this case, we will use the qHNSW index, Euclidean Distance (L2) for the search metric, and we specify the number of dimensions of our embeddings (384).

In [62]:
#Set up the schema and indexes for KDB.AI table, specifying embeddings column with 384 dimensions, Euclidean Distance, and flat index
pdf_schema = [
    {"name": "document_id", "type": "bytes"},
    {"name": "text", "type": "bytes"},
    {"name": "embeddings", "type": "float32s"}
]

indexes = [
    {
        "name": "qhnsw_index",
        "type": "qHnsw",
        "column": "embeddings",
        "params": {"dims": 1536, "metric": "L2"},
    }
]


### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [63]:
# get the database connection. Default database name is 'default'
database = session.database('default')

# First ensure the table does not already exist
try:
    database.table("pdf").drop()
except kdbai.KDBAIException:
    pass

In [64]:
table = database.create_table("pdf", schema = pdf_schema, indexes = indexes)

We can use `query` to see our table exists but is empty.

In [65]:
table.query()

,document_id,text,embeddings


## 3. LlamaIndex index & query_engine setup
Define the index: using KDB.AI as the vector store, chunk, embed, and load the document into KDB.AI

In [66]:
vector_store = KDBAIVectorStore(table)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=2048, chunk_overlap=0)],
)

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [67]:
show_df(table.query())

(22, 3)


,document_id,text,embeddings
0,b'cb35e257-647f-4953-a447-919fd99f8957',"b'Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J. Ted Mackereth\n ,3, 4, 5, \xe2\x88\x97and\nJohn C. Forbes\...","[-0.001896298, 0.04461563, 0.016039172, 0.004387382, 0.020142527, -0.02016926, 0.014515451, -0.014675843, -0.010986833, 0.001757626, -0.019247007, 0.0028486238, -0.006916893, -0.027132934, 0.027560644, 0.03560696, -0.026772052, -0.0022705453, -0.025194867, 0.018832661, 0.0025261696, -0.014903064..."
1,b'f8b82c1f-d988-4f4f-acb5-0fe77d12baa2',"b'2 Hopkins et al.\nInitially it was expected that interstellar objects would display cometary characteristics (e.g. Jewitt 2003). The pop-\nulation\xe2\x80\x99s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the sourc...","[0.004427607, 0.00960885, -0.0025132392, 0.006977855, 0.011768823, -0.024210533, 0.029257199, -0.01263012, 0.016055124, 0.024721928, -0.0077987793, 0.008895588, -0.012596475, -0.040830884, -0.01087388, 0.019419566, -0.03523245, -0.0022962324, -0.031814177, -0.00019114243, 0.015880171, -0.0127579..."
2,b'35284c91-4732-4831-b78d-c2284ec603b5',"b'The Galactic ISO Population 3\nprocesses modelled, and demonstrate this method by constraining the metallicity dependence of the ISO production\nrate.\n2.APOGEE AND STELLAR DENSITY MODELLING\nTo predict the distribution of ISOs in the Milky Way, we first obtain the distribution of all stars th...","[0.039092664, -0.01292948, 0.067334704, 0.008587963, 0.011623856, -0.018899858, 0.009500632, -0.013347787, -0.028723726, -0.0010053621, -0.021295615, 0.014830874, -0.03392087, 0.0021153009, 0.018215355, 0.02143505, -0.011446392, -0.007104876, -0.03916872, 0.032222293, 0.009564012, 0.010742877, 0..."
3,b'6854d718-c425-4db7-a4dc-2737db1538aa',"b'4 Hopkins et al.\nfollows that the probability of finding a point (i.e. an observed star) with observables in the infinitesimal volume \xce\xb4O\nis given by \xce\xbb(O)\xce\xb4O, and the total number of points (i.e. stars observed) is a Poisson random variable with mean and\nvariance \xce\x9b...","[0.023402294, 0.0226368, 0.07654956, -0.010600748, 0.017483374, -0.0025510825, 0.019055374, -0.004674991, -0.0119472, 0.0058847475, -0.0091654435, 0.0012020674, -0.053557355, 0.005966765, 0.03362713, 0.014708452, 0.008085548, -0.014038643, -0.050686747, 0.036224347, -0.010135982, 0.005768556, 0...."
4,b'0033757f-3651-4bdd-a52a-4e20aaa2da24',"b'The Galactic ISO Population 5\nThis particular form for the density profile has the advantage that the Poisson point process likelihood takes the\ntractable form\nlnL(logA , aR, az, \xcf\x840, \xcf\x89) = const + N\x10\nlogA\xe2\x88\x92aR\xe2\x9f\xa8R\xe2\x88\x92R0\xe2\x9f\xa9 \xe2\x88\x92az\x...","[0.0287323, 0.04074224, 0.051109564, 0.010259612, 0.029594, -0.013921836, 0.021421317, 0.0003107252, -0.0040055574, -0.00013253682, -0.015672164, 0.012669679, -0.037403155, 0.008536213, 0.019495957, 0.032125242, -0.033660144, -0.012521574, -0.04103845, -0.014177653, 0.03632603, 0.0049749697, 0.0..."


#### Set up the LlamaIndex Query Engine

In [68]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    vector_store_kwargs={
                    "index" : "qhnsw_index",
                },
)

## 4. Retrieve Similar Sentences & RAG



Now that the embeddings are stored in KDB.AI, we can perform semantic search using through the LlamaIndex query engine.

### Search 1


In [69]:
#search_term1 = "The Milky Way is thought to host a huge population of interstellar objects (ISOs), numbering"
search_term1 = "what is the estimated population of interstellar objects (ISOs) in the milky way"

In [70]:
retrieved_chunks = query_engine.retrieve(search_term1)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

Draft version August 14, 2023
Typeset using L ATEX default style in AASTeX631
The Galactic Interstellar Object Population: A Framework for Prediction and Inference
Matthew J. Hopkins
 ,1Chris Lintott
 ,1Michele T. Bannister
 ,2J. Ted Mackereth
 ,3, 4, 5, ∗and
John C. Forbes
2
1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK
2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand
3Just Group plc, Enterprise House, Bancroft road, Reigate, Surrey RH2 7RP, UK
4Canadian Institute for Theoretical Astrophysics, University of Toronto, 60 St. George Street, Toronto, ON, M5S 3H8, Canada
5Dunlap Institute for Astronomy and Astrophysics, University of Toronto, 50 St. George Street, Toronto, ON M5S 3H4, Canada
ABSTRACT
The Milky Way is thought to host a huge population of interstellar objects (ISOs), numbering
approximately 1015pc−3around the Sun, which are formed and 

We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Now we can perform RAG, passing the retrieved chunks from above to the LLM for a generate response:

In [71]:
result = query_engine.query(search_term1)
print(result.response)

The estimated population of interstellar objects (ISOs) in the Milky Way is approximately 10^15 per cubic parsec around the Sun.


### Search 2

Let's try another search term.

In [72]:
search_term2 = "how does planet formation occur"

In [73]:
retrieved_chunks = query_engine.retrieve(search_term2)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

14 Hopkins et al.
because the probability that two randomly selected ISOs have the same parent star is approximately equal to one over
the total number of stars which could contribute ISOs to the population around the Sun. The ISO sample likelihood
can then be combined with priors to form a posterior distribution on the model parameters:
p(θ|O1, . . . ,ON)∝p(θ)·Y
ip(Oi|θ) (10)
This posterior can be used to calculate estimates and confidence intervals for the values of the parameters θ. An
example of this calculation is given in section 6.3.
5.2. Additional Properties of ISOs
As with all minor planets, there are many observable quantities for ISOs that could be used in this framework in
the set of observable properties O. If the distribution of ISOs in these properties can be predicted, the inference
method of §5.1 can be used to compare the predicted distribution to the observed distribution in these properties, to
make inferences about the models used. If multiple properties are inclu

Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way.

In [74]:
result = query_engine.query(search_term2)
print(result.response)

Planet formation occurs through processes in protoplanetary disks, where dust and gas surrounding a young star coalesce to form solid bodies. Initially, small particles collide and stick together, forming larger aggregates known as planetesimals. These planetesimals can further collide and merge, eventually leading to the formation of protoplanets. The dynamics of this process can be influenced by factors such as the metallicity of the star, which affects the availability of solid material, and the gravitational interactions with giant planets, which can scatter and eject planetesimals into interstellar space. The composition of the resulting bodies is linked to the chemical properties of the protoplanetary disk, which varies with the star's metallicity and the conditions present during formation.


## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [75]:
table.drop()

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/vm88uwcB)